## Apiux & SII: calculo de indice de materia oscura en personas juridicas.
## ATENCION: proyecto sujeto a mantenimiento continuo. 

## Henry Vega (henrry.vega@api-ux.com)
## Data analyst

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
#warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:

spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.yarn.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
  .config("spark.executor.memory", "24g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "12") \
  .config("spark.executor.instances", "24") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()

Setting spark.hadoop.yarn.resourcemanager.principal to yasser.nanjari


## Carga de relaciones societarias(depurada)

In [3]:
Carpeta = "Proceso Renta 2025/"
Archivo = "JBA_MAT_OBS_RENTA_24-2.csv"
Carpeta_Salida = "Renta 2025/" 

Se carga la data depurada anteriormente de relaciones societarias.

In [4]:
df = spark.read.options(header=True,inferSchema=True,delimiter=",").csv(f"../../data/processed/{Carpeta_Salida}sociedades_participacion_capital_nozero.csv")
df.createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad order by RUT_SOCIEDAD asc").show()

Hive Session ID = 599ae2ab-3232-4475-b0d0-e54ac58b7001                          


+------------+---------+------------------+---------------------+------------------------+------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|PERI_AGNO_MES_TRIBUTARIO|FUENTE|
+------------+---------+------------------+---------------------+------------------------+------+
|   318500215|       12|             100.0|                100.0|                  202400|  RIAC|
|   319519583|       12|             100.0|                100.0|                  202400|  RIAC|
|   350003575| 29485615|             100.0|                100.0|                  202400| F1948|
|   350004975| 31161877|             33.33|                33.33|                  202400|  RIAC|
|   350004975| 58631130|             33.33|                33.33|                  202400|  RIAC|
|   350004975| 53717298|             33.33|                33.33|                  202400|  RIAC|
|   350010995| 39429206|              50.0|                 50.0|                  202400| F1947|
|   350010995| 45906

## Exploracion de composiciones de socios en capital y utilidades

Como no hay unicidad de entradas y se puede presentar varias veces una combinacion sociedad y socio, se agrupa por el promedio para cada relacion. De esta forma analizamos la composicion de la sociedad en utilidades y capital.


In [5]:
spark.sql("select  RUT_SOCIEDAD, RUT_SOCIO, mean(PORCENTAJE_CAPITAL) as PORCENTAJE_CAPITAL, mean(PORCENTAJE_UTILIDADES) as PORCENTAJE_UTILIDADES from sociedad group by RUT_SOCIEDAD, RUT_SOCIO").createOrReplaceTempView("composicion")
spark.sql("select  RUT_SOCIEDAD, sum(PORCENTAJE_CAPITAL) as TOTAL_CAPITAL, SUM(PORCENTAJE_UTILIDADES) as TOTAL_UTILIDADES from composicion group by RUT_SOCIEDAD").createOrReplaceTempView("composicion")
m=spark.sql("select * from composicion").toPandas()


Veamos cuantas sociedades suman mas del 100% en capital total.

In [6]:
m[m['TOTAL_CAPITAL']>105] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
37290,534301045,110.00,100.00
61602,532739170,105.98,105.98
74151,542936525,110.00,110.00
87226,532191784,110.00,100.00
95624,534272275,109.00,109.00
...,...,...,...
1412286,541195415,110.00,80.00
1412516,534615345,110.00,110.00
1427349,532532145,109.00,109.00
1432399,532279641,109.00,100.00


Veamos cuantas sociedades suman menos de 90% en capital total.

In [7]:
m[m['TOTAL_CAPITAL']<90] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
7792,677369565,84.53,84.53
14745,600086205,1.17,1.17
14879,493274605,74.10,74.10
14932,589115105,62.58,62.58
15178,543932730,83.86,83.86
...,...,...,...
1383475,455019752,61.02,61.02
1388619,660142005,33.84,33.84
1388867,678095255,68.81,68.81
1410910,676778835,82.10,82.10


In [8]:
m[m['TOTAL_UTILIDADES']>105] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
10653,537126315,100.00,130.00
10813,356934905,100.00,120.00
17440,697122095,100.00,109.00
24727,545546685,100.03,122.83
33420,544877835,100.00,145.00
...,...,...,...
1427349,532532145,109.00,109.00
1438077,532227113,100.00,106.00
1439672,533731364,100.00,108.00
1439715,545957025,100.00,109.00


In [9]:
m[m['TOTAL_UTILIDADES']<95] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
1100,539351545,99.64,83.34
1218,558153685,99.98,41.70
1501,532662275,100.00,50.00
2623,551419825,100.00,40.00
2704,544401485,100.00,5.00
...,...,...,...
1441118,556994485,100.00,50.00
1441556,535064262,93.36,93.36
1442458,535525296,96.13,51.13
1444704,544225225,99.90,0.01


Se utilizara el porcentaje de capital en lugar de utilidades para hacer la transmision de materia oscura. En el proceso de limpieza de datos se considero las filas con valores no nuels de PORCENTAJE_CAPITAL. Se utilizara una tabla de composicion para poder normalizar los valores de oscuridad para cuando los valores de porcentajes se capital sean diferentes al 100%.

## Tabla de ponderacion para el total de capital.

In [10]:
spark.sql("select RUT_SOCIEDAD as CONT_RUT, TOTAL_CAPITAL as ponderador from composicion").createOrReplaceTempView("composicion")

## Lectura de tabla de oscuridad

Se hace lectura de los datos de iniciales de oscuridad para personas naturales.

In [11]:
osc = spark.read.options(header=True,inferSchema=True,delimiter=";").csv(f"../../data/raw/{Carpeta}{Archivo}")
osc.createOrReplaceTempView("oscuridad")


In [12]:
spark.sql('select * from oscuridad').show()

+---------+-----+---------+---------+---------+---------+---------+---------+
| CONT_RUT|Valor|Periodo_6|Periodo_5|Periodo_4|Periodo_3|Periodo_2|Periodo_1|
+---------+-----+---------+---------+---------+---------+---------+---------+
|343103073|  1.0|     null|     null|     null|     null|     null|     null|
|343030602|  1.0|     null|     null|     null|     null|     null|     null|
|337493581|  1.0|     null|     null|     null|     null|     null|     null|
|343178673|  1.0|     null|     null|     null|     null|     null|     null|
|343001874|  1.0|     null|     null|     null|     null|     null|     null|
|337464909|  1.0|     null|     null|     null|     null|     null|     null|
|343066771|  1.0|     null|     null|     null|     null|     null|     null|
|337362828|  1.0|     null|     null|     null|     null|     null|     null|
|343136533|  1.0|     null|     null|     null|     null|     null|     null|
|337531668|  1.0|     null|     null|     null|     null|     nu

## Ajuste de valores de participacion societaria.

Como para una combinacion sociedad socio tenemos distintos valores de PORCENTAJE_CAPITAL y PORCENTAJE_UTILIDADES se usara el promedio de dichos valores.

In [13]:
spark.sql("select  RUT_SOCIEDAD, RUT_SOCIO, mean(PORCENTAJE_CAPITAL) as PORCENTAJE_CAPITAL, mean(PORCENTAJE_UTILIDADES) as PORCENTAJE_UTILIDADES from sociedad group by RUT_SOCIEDAD, RUT_SOCIO").createOrReplaceTempView("sociedad")

## Primera iteracion

Se realiza el cruce de la data societaria con la data de  oscuridad de personas naturales (1re paso de calculo de materia oscura para sociedades completas). Para ello se completa la malla con la oscuridad inicial. Hay sociedades donde no se completan todos los socios con oscuridad, por lo cual se discrimina mediante un contador de nulos de dicho campo para poder agregar la data y obtener un valor de oscuridad para un rut de soiedad donde todas las entradas de sus socios han sido completadas.

In [14]:
#Iteracion 0
spark.sql("select * from sociedad left join oscuridad on sociedad.RUT_SOCIO=oscuridad.CONT_RUT order by sociedad.RUT_SOCIEDAD asc").createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad ").show()
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, PORCENTAJE_CAPITAL, Valor from sociedad").createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad ").show()
spark.sql("select RUT_SOCIEDAD as RUT_SOCIEDAD1, count(*) as nulos  from sociedad where Valor is null group by RUT_SOCIEDAD order by RUT_SOCIEDAD ASC").createOrReplaceTempView("aux")
spark.sql("select * from aux ").show()
spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, nulos, Valor from sociedad left join aux on sociedad.RUT_SOCIEDAD=aux.RUT_SOCIEDAD1 order by RUT_SOCIEDAD asc ").createOrReplaceTempView("aux")
spark.sql("select * from aux where nulos is null ").show()


+------------+---------+------------------+---------------------+--------+-----+---------+---------+---------+---------+---------+---------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|CONT_RUT|Valor|Periodo_6|Periodo_5|Periodo_4|Periodo_3|Periodo_2|Periodo_1|
+------------+---------+------------------+---------------------+--------+-----+---------+---------+---------+---------+---------+---------+
|   318500215|       12|             100.0|                100.0|    null| null|     null|     null|     null|     null|     null|     null|
|   319519583|       12|             100.0|                100.0|    null| null|     null|     null|     null|     null|     null|     null|
|   350003575| 29485615|             100.0|                100.0|29485615|  0.0|        3|        4|        2|        2|        2|        2|
|   350004975| 53717298|             33.33|                33.33|53717298|  0.0|        2|        2|        1|        1|        1|        1|
|   350004975

+------------+---------+------------------+-----+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|Valor|
+------------+---------+------------------+-----+
|   318500215|       12|             100.0| null|
|   319519583|       12|             100.0| null|
|   350003575| 29485615|             100.0|  0.0|
|   350004975| 58631130|             33.33|  0.0|
|   350004975| 31161877|             33.33|  0.0|
|   350004975| 53717298|             33.33|  0.0|
|   350010995| 45906950|              50.0|  0.0|
|   350010995| 39429206|              50.0|  0.0|
|   350016945| 44467071|              50.0|  0.0|
|   350016945| 35843526|              50.0|  0.0|
|   350027445| 31838490|              17.7|  0.0|
|   350027445| 39453818|              58.7|  0.0|
|   350027445| 29543358|              23.6|  0.0|
|   350030315| 28725597|              50.0|  0.0|
|   350030315| 29546039|              50.0|  0.0|
|   350032485| 33678482|             24.59|  0.0|
|   350032485| 55904497|             24.59|  0.0|


+-------------+-----+
|RUT_SOCIEDAD1|nulos|
+-------------+-----+
|    318500215|    1|
|    319519583|    1|
|    350066505|    2|
|    350179485|    2|
|    350255785|    3|
|    350484615|    3|
|    350598645|    1|
|    350638125|    2|
|    350720935|    1|
|    350743615|    1|
|    350823975|    1|
|    351005695|    5|
|    351034045|    1|
|    351113565|    3|
|    351242785|    1|
|    351414705|    1|
|    351451805|    1|
|    351606085|    1|
|    351667685|    1|
|    351679865|    1|
+-------------+-----+
only showing top 20 rows



[Stage 49:=========================================>           (156 + 12) / 200]

+------------+---------+------------------+-----+-----+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|nulos|Valor|
+------------+---------+------------------+-----+-----+
|   350003575| 29485615|             100.0| null|  0.0|
|   350004975| 53717298|             33.33| null|  0.0|
|   350004975| 58631130|             33.33| null|  0.0|
|   350004975| 31161877|             33.33| null|  0.0|
|   350010995| 39429206|              50.0| null|  0.0|
|   350010995| 45906950|              50.0| null|  0.0|
|   350016945| 44467071|              50.0| null|  0.0|
|   350016945| 35843526|              50.0| null|  0.0|
|   350027445| 39453818|              58.7| null|  0.0|
|   350027445| 29543358|              23.6| null|  0.0|
|   350027445| 31838490|              17.7| null|  0.0|
|   350030315| 28725597|              50.0| null|  0.0|
|   350030315| 29546039|              50.0| null|  0.0|
|   350032485| 60665820|             24.59| null|  0.0|
|   350032485| 33678482|             24.59| null

## Primera iteracion, cruce con oscuridad de personas naturales (2do paso de calculo de oscuridad y ponderacion por total de capital)

Junto con completar la data con los valores de oscuridad para sociedades completas, se agrega un ponderador. De esta forma, si una entidad tiene dos socios con oscuridad y la composicion suma diferente a 100%, se pondera proporcionalmente para que sumen 100%.

In [15]:

#agregar a la oscuridad la ponderacion adecuada

spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, Valor from aux where nulos is null").createOrReplaceTempView("aux")
#spark.sql("select * from aux").show()
spark.sql("select RUT_SOCIEDAD as CONT_RUT, SUM(PORCENTAJE_CAPITAL*Valor*0.01) as othervalue from aux group by RUT_SOCIEDAD").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad").show()
spark.sql("select CONT_RUT, othervalue as Value from oscuridad").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad").show()

spark.sql("select oscuridad.CONT_RUT as CONT_RUT, Value, ponderador from oscuridad left join composicion on oscuridad.CONT_RUT=composicion.CONT_RUT order by oscuridad.CONT_RUT desc ").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad ").show()
spark.sql("select CONT_RUT, Value/ponderador*100 as Value from oscuridad ").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad order by Value desc ").show()

#spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_UTILIDADES, Valor from aux where nulos is null").createOrReplaceTempView("aux")

oscuridad_aux=spark.sql("select * from oscuridad ").toPandas()
oscuridad_aux['iterations']=0


In [16]:
oscuridad_aux

,CONT_RUT,Value,iterations
0,697198955,0.0,0
1,697197695,0.0,0
2,697197065,0.0,0
3,697196925,0.0,0
4,697196435,0.0,0
...,...,...,...
1337077,350027445,0.0,0
1337078,350016945,0.0,0
1337079,350010995,0.0,0
1337080,350004975,0.0,0


In [17]:
#soc = spark.sql("SELECT RUT_SOCIEDAD, PORCENTAJE_CAPITAL, Valor FROM sociedad WHERE count(RUT_SOCIO) = 1 GROU").toPandas()
#soc = spark.sql("SELECT * FROM sociedad").toPandas()
soc = spark.sql("SELECT RUT_SOCIEDAD AS CONT_RUT, COUNT(RUT_SOCIO) AS N_SOCIOS FROM sociedad GROUP BY RUT_SOCIEDAD").toPandas()

In [18]:
soc2 = soc[soc["N_SOCIOS"]==1]
soc2

,CONT_RUT,N_SOCIOS
0,318500215,1
1,319519583,1
2,350003575,1
10,350056285,1
28,350244445,1
...,...,...
1446406,697195455,1
1446407,697195665,1
1446413,697196715,1
1446418,697197275,1


In [19]:
import pandas as pd
aux = pd.merge(soc2,oscuridad_aux, on = ["CONT_RUT"])
aux = aux.rename(columns = {"CONT_RUT": "RUT_SOCIEDAD"})
aux

,RUT_SOCIEDAD,N_SOCIOS,Value,iterations
0,350003575,1,0.0,0
1,350056285,1,0.0,0
2,350244445,1,0.0,0
3,350385635,1,0.0,0
4,350475375,1,0.0,0
...,...,...,...,...
725861,697181735,1,0.0,0
725862,697183555,1,0.0,0
725863,697191745,1,0.0,0
725864,697193425,1,0.0,0


In [20]:
dataframe = osc.toPandas().rename(columns = {"CONT_RUT": "RUT_SOCIO"})
dataframe

,RUT_SOCIO,Valor,Periodo_6,Periodo_5,Periodo_4,Periodo_3,Periodo_2,Periodo_1
0,343103073,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,343030602,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,337493581,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,343178673,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,343001874,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
26624588,349999942,0.734137,NaN,1.0,1.0,NaN,NaN,NaN
26624589,349999949,0.734137,NaN,1.0,1.0,NaN,NaN,NaN
26624590,349999956,0.734137,NaN,1.0,1.0,NaN,NaN,NaN
26624591,349999963,0.854984,NaN,NaN,1.0,NaN,NaN,NaN


In [21]:
df1 = df.toPandas()
df1

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES,PERI_AGNO_MES_TRIBUTARIO,FUENTE
0,318500215,12,100.00,100.00,202400,RIAC
1,319519583,12,100.00,100.00,202400,RIAC
2,350003575,29485615,100.00,100.00,202400,F1948
3,350004975,31161877,33.33,33.33,202400,RIAC
4,350004975,53717298,33.33,33.33,202400,RIAC
...,...,...,...,...,...,...
2528397,697198885,75038731,20.00,20.00,202400,RIAC
2528398,697198885,697168365,80.00,80.00,202400,RIAC
2528399,697198955,53430011,99.90,99.90,202400,RIAC
2528400,697199025,75474502,1.00,1.00,202400,F1948


In [22]:
aux2 = pd.merge(pd.merge(df1, aux, on = ["RUT_SOCIEDAD"]), dataframe, on = ["RUT_SOCIO"])
aux2 = aux2.rename(columns = {"Value":"Oscuridad_PJ","Valor":"Oscuridad_PN"})
aux2 = aux2.sort_values(by=["Oscuridad_PN"])


In [23]:
aux2[aux2["Oscuridad_PN"]>0].head(30)

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES,PERI_AGNO_MES_TRIBUTARIO,FUENTE,N_SOCIOS,Oscuridad_PJ,iterations,Oscuridad_PN,Periodo_6,Periodo_5,Periodo_4,Periodo_3,Periodo_2,Periodo_1
354348,539008671,337476263,100.0,100.0,202400,RIAC,1,0.100706,0,0.100706,NaN,2.0,1.0,1.0,1.0,1.0
698091,544654346,182481906,100.0,100.0,202400,RIAC,1,0.100706,0,0.100706,NaN,2.0,3.0,3.0,3.0,6.0
695063,544600691,185345102,100.0,100.0,202400,F1948,1,0.100706,0,0.100706,NaN,4.0,2.0,2.0,6.0,6.0
698127,544654864,142301185,100.0,100.0,202400,F1948,1,0.100706,0,0.100706,NaN,1.0,1.0,4.0,1.0,4.0
520559,541832709,187598381,100.0,100.0,202400,RIAC,1,0.100706,0,0.100706,NaN,1.0,2.0,3.0,4.0,5.0
360052,539104326,73679128,100.0,100.0,202400,RIAC,1,0.100706,0,0.100706,NaN,3.0,2.0,2.0,2.0,1.0
359800,539100280,138167496,100.0,100.0,202400,RIAC,1,0.100706,0,0.100706,NaN,5.0,3.0,4.0,5.0,5.0
426753,543142101,70532439,100.0,100.0,202400,RIAC,1,0.100706,0,0.100706,NaN,5.0,4.0,5.0,5.0,4.0
607270,544653681,144678805,100.0,100.0,202400,RIAC,1,0.100706,0,0.100706,NaN,3.0,3.0,2.0,4.0,4.0
678671,544308126,187913906,100.0,100.0,202400,F1948,1,0.100706,0,0.100706,NaN,3.0,4.0,4.0,5.0,8.0


In [24]:
aux2[aux2["RUT_SOCIEDAD"]== "XYtAg9XPdlsmKvL99k3t/w=="].head()
#aux2[aux2["RUT_SOCIO"]== "eK5KCE9n7ek2CyHGijVAAQ=="].head()

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES,PERI_AGNO_MES_TRIBUTARIO,FUENTE,N_SOCIOS,Oscuridad_PJ,iterations,Oscuridad_PN,Periodo_6,Periodo_5,Periodo_4,Periodo_3,Periodo_2,Periodo_1


## Primera iteracion,resultados

Principales resultados de la primera iteracion, es decir la iteracion cero. 

In [25]:
oscuridad_aux.describe()

,CONT_RUT,Value,iterations
count,1.337082e+06,1.337082e+06,1337082.0
mean,5.381797e+08,1.560652e-02,0.0
std,2.675574e+07,6.977926e-02,0.0
min,3.500036e+08,0.000000e+00,0.0
25%,5.356931e+08,0.000000e+00,0.0
50%,5.392709e+08,0.000000e+00,0.0
75%,5.424603e+08,0.000000e+00,0.0
max,6.971990e+08,1.000000e+00,0.0


## Iteraciones subsiguientes

Se repite el proceso iterativo para asi completar mas sociedades. El parametro *iter* hara referencia al numero de iteraciones luego de la iteracion 0.

In [26]:
iter=6

In [27]:
for iteration in range(1,iter):
    
    spark.sql("select * from sociedad left join oscuridad on sociedad.RUT_SOCIO=oscuridad.CONT_RUT order by sociedad.RUT_SOCIEDAD asc").createOrReplaceTempView("sociedad")
    spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, PORCENTAJE_CAPITAL, CASE WHEN Valor is null THEN Value ELSE Valor END AS Valor from sociedad").createOrReplaceTempView("sociedad")
    spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, PORCENTAJE_CAPITAL, Valor from sociedad").createOrReplaceTempView("sociedad")
    spark.sql("select RUT_SOCIEDAD as RUT_SOCIEDAD1, count(*) as nulos from sociedad where Valor is null group by RUT_SOCIEDAD order by RUT_SOCIEDAD ASC").createOrReplaceTempView("aux")
    spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, nulos, Valor from sociedad left join aux on sociedad.RUT_SOCIEDAD=aux.RUT_SOCIEDAD1 order by RUT_SOCIEDAD asc ").createOrReplaceTempView("aux")
    spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, Valor from aux where nulos is null").createOrReplaceTempView("aux")
    spark.sql("select RUT_SOCIEDAD as CONT_RUT, SUM(PORCENTAJE_CAPITAL*Valor*0.01) as othervalue from aux group by RUT_SOCIEDAD").createOrReplaceTempView("oscuridad")
    spark.sql("select CONT_RUT, othervalue as Value from oscuridad").createOrReplaceTempView("oscuridad")
    
    spark.sql("select oscuridad.CONT_RUT as CONT_RUT, Value,ponderador from oscuridad left join composicion on oscuridad.CONT_RUT=composicion.CONT_RUT order by oscuridad.CONT_RUT asc ").createOrReplaceTempView("oscuridad")
    #spark.sql("select * from oscuridad ").show()
    spark.sql("select CONT_RUT, Value/ponderador*100 as Value from oscuridad order by Value desc ").createOrReplaceTempView("oscuridad")
    #spark.sql("select * from oscuridad ").show()    
    oscuridad=spark.sql("select * from oscuridad ").toPandas()
    oscuridad['iterations']=iteration

    oscuridad=oscuridad.merge(oscuridad_aux, on = "CONT_RUT", how = "left")
    oscuridad['iterations']=oscuridad[["iterations_x", "iterations_y"]].min(axis=1)
    oscuridad = oscuridad.rename(columns={'Value_x': 'Value'})
    oscuridad=oscuridad[['CONT_RUT','Value','iterations']]
    oscuridad_aux=oscuridad.iloc[:,:]
    print(oscuridad_aux.describe())

           CONT_RUT         Value    iterations
count  1.383007e+06  1.383007e+06  1.383007e+06
mean   5.385449e+08  1.520639e-02  3.320663e-02
std    2.733875e+07  6.882242e-02  1.791758e-01
min    3.500036e+08  0.000000e+00  0.000000e+00
25%    5.356279e+08  0.000000e+00  0.000000e+00
50%    5.392381e+08  0.000000e+00  0.000000e+00
75%    5.424728e+08  0.000000e+00  0.000000e+00
max    6.971990e+08  1.000000e+00  1.000000e+00


           CONT_RUT         Value    iterations
count  1.395928e+06  1.395928e+06  1.395928e+06
mean   5.386868e+08  1.509519e-02  5.141168e-02
std    2.759068e+07  6.854172e-02  2.593858e-01
min    3.500036e+08  0.000000e+00  0.000000e+00
25%    5.356135e+08  0.000000e+00  0.000000e+00
50%    5.392314e+08  0.000000e+00  0.000000e+00
75%    5.424793e+08  0.000000e+00  0.000000e+00
max    6.971990e+08  1.000000e+00  2.000000e+00


           CONT_RUT         Value    iterations
count  1.401456e+06  1.401456e+06  1.401456e+06
mean   5.387625e+08  1.504639e-02  6.304229e-02
std    2.773600e+07  6.841766e-02  3.180790e-01
min    3.500036e+08  0.000000e+00  0.000000e+00
25%    5.356077e+08  0.000000e+00  0.000000e+00
50%    5.392281e+08  0.000000e+00  0.000000e+00
75%    5.424826e+08  0.000000e+00  0.000000e+00
max    6.971990e+08  1.000000e+00  3.000000e+00


           CONT_RUT         Value    iterations
count  1.403782e+06  1.403782e+06  1.403782e+06
mean   5.387938e+08  1.502584e-02  6.956564e-02
std    2.779726e+07  6.836420e-02  3.558739e-01
min    3.500036e+08  0.000000e+00  0.000000e+00
25%    5.356051e+08  0.000000e+00  0.000000e+00
50%    5.392269e+08  0.000000e+00  0.000000e+00
75%    5.424838e+08  0.000000e+00  0.000000e+00
max    6.971990e+08  1.000000e+00  4.000000e+00


           CONT_RUT         Value    iterations
count  1.404848e+06  1.404848e+06  1.404848e+06
mean   5.388056e+08  1.501803e-02  7.330686e-02
std    2.781826e+07  6.834074e-02  3.807650e-01
min    3.500036e+08  0.000000e+00  0.000000e+00
25%    5.356034e+08  0.000000e+00  0.000000e+00
50%    5.392257e+08  0.000000e+00  0.000000e+00
75%    5.424838e+08  0.000000e+00  0.000000e+00
max    6.971990e+08  1.000000e+00  5.000000e+00


## Resultados y guardado en archivo csv

Luego de estas iteraciones, se obtienen dos outputs. En primer lugar una tabla actualizada de sociedades y socios con cada valor calculado (o no) de la oscuridad de sus socios. 
En segundo lugar la oscuridad calculada para sociedades.

In [28]:
sociedad=spark.sql("select * from sociedad").toPandas()

In [29]:
# Se guardan los archivos finales.
sociedad.to_csv(f'../../data/processed/{Carpeta_Salida}Sociedades_oscuridad_actualizada.csv', index=False)
oscuridad_aux.to_csv(f'../../data/processed/{Carpeta_Salida}Sociedades_oscuridad_completa.csv', index=False)
